# Data Prep

In [60]:
import pandas as pd
import matplotlib as mlt
import numpy as np

# load labor force data

df1 = pd.read_csv("/workspace/DS4002Project2/DATA/laborforce/laborforce.csv")
print( df1.columns )

# we need the columns to be separated by commas. Let's do that here:

input_file = '/workspace/DS4002Project2/DATA/laborforce/laborforce.csv'
output_file = '/workspace/DS4002Project2/DATA/laborforce/laborforce_new.csv'

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        cleaned_line = ','.join(line.split())
        outfile.write(cleaned_line + '\n')
print(f"Processed file saved as {output_file}.")

df = pd.read_csv( "/workspace/DS4002Project2/DATA/laborforce/laborforce_new.csv" )

# Split "Year" column of the df into year, month, and day variables
df[['Year', 'Month', 'Day']] = df['DATE'].str.split('-', expand=True)
print(df.head()) # printing head -- we see that this successfully created new columns. We will use the "Year" column for merging

# rename column to specify variable for female labor force participation rate:
df = df.rename(columns={'LNS11300002': 'Female_LFPR'})
print(df.head())

# create version of the dataframe which only includes the first row for each year recorded, because we don't want duplicates
df1_unique = df.drop_duplicates(subset='Year', keep='first')
print(df1_unique.head()) # looks like this worked

Index(['DATE\tLNS11300002'], dtype='object')
Processed file saved as /workspace/DS4002Project2/DATA/laborforce/laborforce_new.csv.
         DATE  LNS11300002  Year Month Day
0  1948-01-01         32.0  1948    01  01
1  1948-02-01         32.4  1948    02  01
2  1948-03-01         32.1  1948    03  01
3  1948-04-01         33.0  1948    04  01
4  1948-05-01         32.0  1948    05  01
         DATE  Female_LFPR  Year Month Day
0  1948-01-01         32.0  1948    01  01
1  1948-02-01         32.4  1948    02  01
2  1948-03-01         32.1  1948    03  01
3  1948-04-01         33.0  1948    04  01
4  1948-05-01         32.0  1948    05  01
          DATE  Female_LFPR  Year Month Day
0   1948-01-01         32.0  1948    01  01
12  1949-01-01         32.6  1949    01  01
24  1950-01-01         33.4  1950    01  01
36  1951-01-01         34.3  1951    01  01
48  1952-01-01         35.0  1952    01  01


In [61]:

# remove commas from pay equity data and save as new csv dataset

pay_commas = '/workspace/DS4002Project2/DATA/pay_equity/pay_equity.csv'
pay_new = '/workspace/DS4002Project2/DATA/pay_equity/pay_equity_new.csv'

with open(pay_commas, 'r') as file:
    file_data = file.read()
file_data = file_data.replace(',', '')
with open(pay_new, 'w') as file:
    file.write(file_data)

print(f"All commas removed and saved to {pay_new}.")

df2 = pd.read_csv("/workspace/DS4002Project2/DATA/pay_equity/pay_equity_new.csv")
print( df2.columns ) # this shows that there are many spaces in our csv file, which need to be replaced with commas. Let's perform more cleaning:

# replace all spaces in "pay_equity_new.csv" with commas, and save file as "pay_equity_new2.csv"

input_file = '/workspace/DS4002Project2/DATA/pay_equity/pay_equity_new.csv'
output_file = '/workspace/DS4002Project2/DATA/pay_equity/pay_equity_new2.csv'

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        cleaned_line = ','.join(line.split())
        outfile.write(cleaned_line + '\n')
print(f"Processed file saved as {output_file}.")

df2 = pd.read_csv("/workspace/DS4002Project2/DATA/pay_equity/pay_equity_new2.csv")

# rename "percent" column to "wage ratio". This column shows the percentage of a dollar that women receive for each dollar earned by a man
df2 = df2.rename(columns={'Percent': 'Wage_ratio'})

print( len(df2.columns) )


All commas removed and saved to /workspace/DS4002Project2/DATA/pay_equity/pay_equity_new.csv.
Index(['Year   Women's         Men's   Dollar        Percent'], dtype='object')
Processed file saved as /workspace/DS4002Project2/DATA/pay_equity/pay_equity_new2.csv.
5


In [62]:
# clean education csv file.

input_file = '/workspace/DS4002Project2/DATA/education_female/education_female.csv'
output_file = '/workspace/DS4002Project2/DATA/education_female/education_female_new.csv'
def process_line(line):
    return ','.join(line.lstrip().split())

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        cleaned_line = process_line(line)
        outfile.write(cleaned_line + '\n')

df3 = pd.read_csv( "/workspace/DS4002Project2/DATA/education_female/education_female_new.csv" )

# looking at the dataset, there are some weird slashes occuring with some of the values. Let's remove those:
df3['Year'] = df3['Year'].str.replace(r'\\4\\', '', regex=True)
print(df3.head()) # printing out the head, the values look normal now
print(df3.columns)
print(f"Number of columns: {len(df3.columns)}")

   Year  Bachelor_percentage Std_error
0  1940                  3.8     (---)
1  1950                  5.2     (---)
2  1960                  5.8     (---)
3  1970                  8.2     (---)
4  1980                 13.6    (0.20)
Index(['Year', 'Bachelor_percentage', 'Std_error'], dtype='object')
Number of columns: 3


In [58]:
# Merge the datasets on the "Year" variable

# select the columns that we want to include from each dataframe:
labor_df = df_unique[ ["Year", "Female_LFPR"]]
education_df = df3[ ["Year", "Bachelor_percentage"] ]
pay_df = df2[ ["Year", "Wage_ratio"] ]

merged_df = pd.merge(labor_df, education_df, on='Year')
merged_df = pd.merge(merged_df, pay_df, on='Year')

# print head of merged dataset:
print( merged_df.head() )

# save the merged dataframe to a csv file:
merged_df.to_csv("/workspace/DS4002Project2/DATA/merged.csv", index=False)



   Year  Female_LFPR  Bachelor_percentage Wage_ratio
0  1960         37.0                  5.8      60.7%
1  1970         43.3                  8.2      59.4%
2  1980         51.6                 13.6      60.2%
3  1990         57.7                 18.4      71.6%
4  1995         58.8                 20.2      71.4%
